**ANALYSYS**


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import math


Before normalizing the data, I just wanted to check it.

I did the following:


In [ ]:
df_train = pd.read_csv("/kaggle/input/scv-data/df_train.csv")

In [ ]:
cols = [
    'index_id', 'id', 
    'reactivity_error', 'deg_error_Mg_pH10', 'deg_error_pH10', 'deg_error_Mg_50C', 'deg_error_50C', 'reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C' 
]


In [ ]:
for c in cols:
    print(c, df_train[c].min(), df_train[c].max()) 

We can see that the labels are symetrical with respect to zero.

In order to check further I took the example of label *reactivity* only. The others have a similar behaviour.

REACTIVITY

Here is the graph of the sorted values of label *reactivity* : 


In [ ]:
x_reactivity = df_train['reactivity'].sort_values().values
plt.plot(x_reactivity, '-o')
plt.show()

We can see that the huge absolute values are rare:

In [ ]:
d_plus_10 = df_train[df_train['reactivity']>10]
print(d_plus_10.shape)

In [ ]:
d_minus_10 = df_train[df_train['reactivity']<-10]
print(d_minus_10.shape)

Now let us see the graph for the reactivity error. We can note several huge values, greater than 10000 :  

In [ ]:
x_reactivity = df_train['reactivity_error'].sort_values().values
plt.plot(x_reactivity, '-o')
plt.show()

If we put them together, it is difficult to see, but at least we see that the huge error values correspond to huge abosulte values in the label.

In [ ]:
df = df_train.loc[:,['index_id','reactivity', 'reactivity_error','position']]
df1=df.copy()
#df1 = df[df['reactivity_error']<10000]

x_reactivity = df1.sort_values(by=['reactivity'])

l = x_reactivity.shape[0]
x = np.arange(1,l+1)
y_r = x_reactivity['reactivity']
y_r_err = x_reactivity['reactivity_error']

fig, ax = plt.subplots()

line2 = ax.plot(x, y_r_err, 'o', label='reactivity error')
line1 = ax.plot(x, y_r, 'o', label='reactivity')

ax.legend()
plt.show()

Let us exclude the huge error values and see what we get. We can see huge error close to zero.

In [ ]:
df = df_train.loc[:,['index_id','reactivity', 'reactivity_error','position']]
df1=df.copy()
df1 = df[df['reactivity_error']<10000]

x_reactivity = df1.sort_values(by=['reactivity'])

l = x_reactivity.shape[0]
x = np.arange(1,l+1)
y_r = x_reactivity['reactivity']
y_r_err = x_reactivity['reactivity_error']

fig, ax = plt.subplots()

line2 = ax.plot(x, y_r_err, 'o', label='reactivity error')
line1 = ax.plot(x, y_r, 'o', label='reactivity')

ax.legend()
plt.show()

Let us see at which point the huge errors are related to zero value in labels.
We can see a correlation. 

The percentage is calculated from the basis : 

Error > 10000
Error < 10000

In [ ]:
df_lt_tt = df[df['reactivity_error']<=10000]
df_gt_tt = df[df['reactivity_error']>10000]

df_r_eqz = df[df['reactivity']==0]
df_r_neqz = df[df['reactivity']!=0]

lt_tt_eqz = df_lt_tt[df_lt_tt['reactivity']==0].shape[0] / df_lt_tt.shape[0]
lt_tt_neqz = df_lt_tt[df_lt_tt['reactivity']!=0].shape[0]  / df_lt_tt.shape[0]
gt_tt_eqz = df_gt_tt[df_gt_tt['reactivity']==0].shape[0]  / df_gt_tt.shape[0]
gt_tt_neqz = df_gt_tt[df_gt_tt['reactivity']!=0].shape[0]  / df_gt_tt.shape[0]

ar = [[lt_tt_eqz, gt_tt_eqz],[lt_tt_neqz, gt_tt_neqz]]
df_cm = pd.DataFrame(ar, index = ['Reactivity==0','Reactivity != 0'],
                  columns = ['Error < 10000','Error > 10000'])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)